In [17]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import datetime

# Méthode pour le calcul des erreurs


In [10]:
def error_p(p,n=1000,Za=1.96):
    return np.sqrt(p*(1-p)/n)*Za
error_p(0.5,int(1900/9))

def margin_p(p,n=1000,Za=1.96):
    er=error_p(p,n,Za)
    return np.array([p-er,p+er])

def error_in_margin(p,n,variation,Za=1.96,double_margin=True):
    # Compute old values
    old_p=p-variation
    # Compute err
    err=error_p(p/100,n,Za)*100
    # Compute err old values
    old_err=error_p(old_p/100,n,Za)*100
    if double_margin:
        return (np.abs(variation)<=err+old_err) & variation!=0
    else:
        return (np.abs(variation)<=err) & variation!=0

def extract_in_margin(fav,defav,evol_fav,evol_defav,pop_size,double_margin=True):
    # Suppression des valeurs sans variation
    fav=fav[evol_fav!=0]
    defav=defav[evol_defav!=0]
    pop_size_fav=pop_size[evol_fav!=0]
    pop_size_defav=pop_size[evol_defav!=0]
    evol_fav=evol_fav[evol_fav!=0]
    evol_defav=evol_defav[evol_defav!=0]

    # Calcul des erreurs
    err_fav=error_p(fav/100,pop_size_fav)*100
    err_defav=error_p(defav/100,pop_size_defav)*100

    # Calcul des anciennes valeurs
    old_fav=fav-evol_fav
    old_defav=defav-evol_defav
    # Calcul des erreurs anciennes valeurs
    err_old_fav=error_p(old_fav/100,pop_size_fav)*100
    err_old_defav=error_p(old_defav/100,pop_size_defav)*100
    if double_margin:
        total_fav_in_margin=np.sum(err_fav+err_old_fav>=np.abs(evol_fav))
        total_defav_in_margin=np.sum(err_defav+err_old_defav>=np.abs(evol_defav))
    else:
        total_fav_in_margin=np.sum(err_fav>=np.abs(evol_fav))
        total_defav_in_margin=np.sum(err_defav>=np.abs(evol_defav))
    total_in_margin=total_fav_in_margin+total_defav_in_margin
    total_lines=len(fav)+len(defav)

    return (total_in_margin,total_lines)

# Load all dataframes



## Load method

In [49]:
def load_data(filepath):
    filenames=[f.rstrip('.p') for f in os.listdir(filepath)]
    df=pd.read_pickle("{}/{}.p".format(filepath,filenames[0]))
    all_data=pd.DataFrame(columns=df.columns)
    all_data.Base=all_data.Base.astype(int)
    for fn in filenames:
        df=pd.read_pickle("{}/{}.p".format(filepath,fn))
        if df.Base.dtype==object:
            df.Base=df.Base.astype(int)
        if 'Date' not in df.columns:
            df['Date']=fn
        all_data=pd.concat([all_data,df],ignore_index=True)
        in_margin,nb_lines=extract_in_margin(df['ST Favorable'],df['ST Défavorable'],df['Evolution ST Favorable'],df['Evolution ST Défavorable'],df.Base)
    all_data.Date=all_data.Date.astype('datetime64')
    return all_data

def load_data_perso(filepath):
    filenames=[f.rstrip('.p') for f in os.listdir(filepath)]
    df=pd.read_pickle("{}/{}.p".format(filepath,filenames[0]))
    all_data=pd.DataFrame(columns=df.columns)
    all_data.Base=all_data.Base.astype(int)
    for fn in filenames:
        df=pd.read_pickle("{}/{}.p".format(filepath,fn))
        if df.Base.dtype==object:
            df.Base=df.Base.astype(int)
        all_data=pd.concat([all_data,df],ignore_index=True)
    all_data.rename(columns={'Opinion positive':'ST Favorable'},inplace=True)
    all_data.Date=all_data.Date.astype('datetime64')
    return all_data


## Load Ipsos


In [39]:
ipsos=load_data('ipsos/data')

,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable,Date
0,Emmanuel MACRON,ENSEMBLE,ENSEMBLE,971.0,3.6,26.3,30.0,29.1,17.2,46.3,23.8,1.4,-2.5,2018-12-08
1,Emmanuel MACRON,Sexe,Homme,462.0,5.2,29.7,34.9,31.6,19.4,51.0,14.1,1.7,-1.1,2018-12-08
2,Emmanuel MACRON,Sexe,Femme,509.0,2.2,23.3,25.5,26.8,15.2,42.0,32.5,1.1,-3.8,2018-12-08
3,Emmanuel MACRON,Age,18 à 24 ans,103.0,7.5,14.8,22.3,22.8,10.9,33.8,44.0,3.4,-0.9,2018-12-08
4,Emmanuel MACRON,Age,25 à 34 ans,154.0,0.2,13.1,13.3,26.2,18.2,44.4,42.3,-8.9,-3.2,2018-12-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36115,Laurent WAUQUIEZ,VotePrésidentielle20171ertour,STNonexpr,238.0,1.4,10.4,11.9,24.3,19.1,43.5,44.7,1.8,-1.1,2021-01-23
36116,Laurent WAUQUIEZ,VotePrésidentielle20172ndtour,EmmanuelMacron,410.0,3.3,17.8,21.1,31.5,30.0,61.5,17.3,-1.0,3.7,2021-01-23
36117,Laurent WAUQUIEZ,VotePrésidentielle20172ndtour,MarineLePen,210.0,2.8,24.1,26.9,25.6,22.4,48.0,25.1,1.1,-5.5,2021-01-23
36118,Laurent WAUQUIEZ,VotePrésidentielle20172ndtour,STExpr,620.0,3.1,20.0,23.1,29.5,27.4,56.9,20.0,-0.3,0.6,2021-01-23


In [24]:
ipsos.head()

,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable,Date
0,Emmanuel MACRON,ENSEMBLE,ENSEMBLE,971,3.6,26.3,30.0,29.1,17.2,46.3,23.8,1.4,-2.5,2018-12-08
1,Emmanuel MACRON,Sexe,Homme,462,5.2,29.7,34.9,31.6,19.4,51.0,14.1,1.7,-1.1,2018-12-08
2,Emmanuel MACRON,Sexe,Femme,509,2.2,23.3,25.5,26.8,15.2,42.0,32.5,1.1,-3.8,2018-12-08
3,Emmanuel MACRON,Age,18 à 24 ans,103,7.5,14.8,22.3,22.8,10.9,33.8,44.0,3.4,-0.9,2018-12-08
4,Emmanuel MACRON,Age,25 à 34 ans,154,0.2,13.1,13.3,26.2,18.2,44.4,42.3,-8.9,-3.2,2018-12-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36115,Laurent WAUQUIEZ,VotePrésidentielle20171ertour,STNonexpr,238,1.4,10.4,11.9,24.3,19.1,43.5,44.7,1.8,-1.1,2021-01-23
36116,Laurent WAUQUIEZ,VotePrésidentielle20172ndtour,EmmanuelMacron,410,3.3,17.8,21.1,31.5,30.0,61.5,17.3,-1.0,3.7,2021-01-23
36117,Laurent WAUQUIEZ,VotePrésidentielle20172ndtour,MarineLePen,210,2.8,24.1,26.9,25.6,22.4,48.0,25.1,1.1,-5.5,2021-01-23
36118,Laurent WAUQUIEZ,VotePrésidentielle20172ndtour,STExpr,620,3.1,20.0,23.1,29.5,27.4,56.9,20.0,-0.3,0.6,2021-01-23


## Load Ifop


In [42]:
ifop=load_data('ifop/data')
ifop.head()

,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable,Date
0,Emmanuel MACRON,ENSEMBLE,ENSEMBLE,1928.0,5.0,22.0,27.0,32.0,40.0,72.0,1.0,4.0,-4.0,2019-01-19
1,Edouard PHILIPPE,ENSEMBLE,ENSEMBLE,1928.0,4.0,26.0,30.0,36.0,31.0,67.0,3.0,-1.0,1.0,2019-01-19
2,Emmanuel MACRON,ENSEMBLE,ENSEMBLE,1891.0,5.0,23.0,28.0,34.0,37.0,71.0,1.0,1.0,-1.0,2019-02-16
3,Edouard PHILIPPE,ENSEMBLE,ENSEMBLE,1891.0,3.0,28.0,31.0,36.0,30.0,66.0,3.0,1.0,-1.0,2019-02-16
4,Emmanuel MACRON,ENSEMBLE,ENSEMBLE,1929.0,4.0,25.0,29.0,36.0,33.0,69.0,2.0,1.0,-2.0,2019-03-23


## Load Ifop personalités

In [53]:
ifop_pers=load_data_perso('ifop/data_perso')
ifop_pers.head()

,Nom,Base,Date,ST Favorable
0,Nicolas HULOT,1008.0,2020-01-11,72
1,Martine AUBRY,1008.0,2020-01-11,45
2,Nicolas SARKOZY,1008.0,2020-01-11,44
3,Edouard PHILIPPE,1008.0,2020-01-11,44
4,Ségolène ROYAL,1008.0,2020-01-11,44
